In [2]:
#pip install pickle-mixin
#pip install fuzzywuzzy
#pip install pyarrow

In [3]:
import pandas as pd
import os
import pickle
import json
from fuzzywuzzy import fuzz

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Business: Contiene información del comercio, incluyendo localización, atributos y categorías.


In [34]:
# Ruta al archivo .pkl que deseas abrir
ruta_archivo_pkl = '../PROYECTO_FINAL_EQUIPO/Yelp/business.pkl'

# Abrir el archivo .pkl en modo lectura binaria
with open(ruta_archivo_pkl, 'rb') as archivo_pkl:
    # Cargar el objeto desde el archivo .pkl
    business_Yelp = pickle.load(archivo_pkl)


In [35]:
business_Yelp.shape

(150346, 28)

In [6]:
business_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

In [36]:
#Eliminando Columnas Duplicadas
business_Yelp = business_Yelp.loc[:, ~business_Yelp.columns.duplicated()]


In [37]:
business_Yelp.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
21415,gVPEtTnLDLaSGyPHXikmWA,Penn Station East Coast Subs,3824 Hampton Ave,Saint Louis,PA,63109,38.594385,-90.293659,2.5,42,1,"{'OutdoorSeating': 'False', 'Caters': 'True', ...","Sandwiches, Fast Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '10:30-22:0',..."
112967,V0dKk_r_LWY63eSumS3hHg,Taste of Greece,709 Dodecanese Blvd,Tarpon Springs,FL,34689,28.155496,-82.760372,4.0,27,1,"{'RestaurantsDelivery': 'False', 'RestaurantsP...","Bakeries, Restaurants, Food",None
32605,iP5pLqJzTUz6ZkVtXzYjHA,"Ruby & Rochon, DDS","15 E Arrellaga St, Ste 4",Santa Barbara,TN,93101,34.427799,-119.709221,4.5,27,1,"{'ByAppointmentOnly': 'True', 'BusinessAccepts...","Health & Medical, Pediatric Dentists, Dentists","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."


Dejar solo los negocios que sean del tipo Restaurantes.

In [41]:
#Generar una lista de categorías únicas:

# Rellena los valores nulos con una cadena vacía
business_Yelp['categories'].fillna('', inplace=True)

# Dividir la columna "categories" en listas de categorías
categories_lists = business_Yelp['categories'].str.split(', ')

# Unir todas las listas en una única lista
all_categories = [category for categories_list in categories_lists for category in categories_list]

# Convertir el conjunto de nuevo en una lista si es necesario
unique_categories = set(all_categories)
unique_categories_list = list(unique_categories)

In [48]:
#Con las categorías únicas y nuestra palabra clave, obtendremos similitud de categorías
#Para asegurar seleccionar todos los restaurantes
keyword = "restaurant"

# Función para calcular la similitud entre la palabra clave y cada categoría
def calcular_similitud(categoria, keyword):
    return fuzz.ratio(keyword, categoria)

# Calcular la similitud para cada categoría en la lista
similitudes = [(categoria, calcular_similitud(categoria, keyword)) for categoria in unique_categories_list]

# Filtrar categorías que tienen una similitud mayor que un umbral (por ejemplo, 70)
umbral_similitud = 60  # Puedes ajustar este valor según tus necesidades
categorias_similares = [categoria for categoria, similitud in similitudes if similitud >= umbral_similitud]

# Ahora, categorias_similares contendrá las categorías similares a "restaurants"
print(categorias_similares

['Vegetarian', 'Pop-Up Restaurants', 'Restaurant Supplies', 'Restaurants']


In [11]:
df_business_Yelp = business_Yelp
# Primero, dividimos los valores de 'categories' en listas de strings
df_business_Yelp['categories'] = df_business_Yelp['categories'].str.split(',')

df_business_Yelp = df_business_Yelp.dropna(subset=['categories'])

#Filtrar por categoría "Restaurantes"
df_business_Yelp = df_business_Yelp[df_business_Yelp['categories'].apply(lambda categories: 'Restaurants' in categories)]

# Unir los valores de la lista en categorías en un solo string separado por comas
df_business_Yelp['categories'] = df_business_Yelp['categories'].apply(lambda x: ','.join(x))

# Recetar el Indice
df_business_Yelp.reset_index(inplace=True, drop=True)

In [49]:
df_business_Yelp.shape

(15290, 14)

## DataSet Review

In [13]:
# Ruta de la carpeta de datos de Google Maps
yelp_folder = '/Users/alejandrobenjaminnunezguzman/Desktop/PROYECTO_FINAL_EQUIPO/Yelp'

# Ruta de la subcarpeta "metadata-sitios"
yelp_subfolder = os.path.join(yelp_folder, 'Reviews')

In [14]:
reviews = []

# Recorrer todos los archivos
for archivo in os.listdir(yelp_subfolder):
        # Ruta del archivo JSON actual
        ruta_archivo = os.path.join(yelp_subfolder, archivo)

        # Leer el contenido del archivo
        with open(ruta_archivo) as f:
            contenido_json = f.readlines()

        # Cargar cada objeto JSON por separado
        for linea in contenido_json:
            try:
                data = json.loads(linea)
            except json.JSONDecodeError as e:
                print(f"Error al cargar el archivo JSON {ruta_archivo}: {e}")
                continue

            reviews.append(data)

# Convertir los resultados a un pandas DataFrame
df_reviews = pd.DataFrame(reviews)

In [15]:
df_reviews.shape

(6990280, 9)

Seleccionamos únicamente reviews de negocios del tipo: Restaurantes

In [17]:
df_business_Yelp.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
5233,1lwulkAD5htni3rbHW8oeg,Subway,"1940 Cliff Dr, Ste B-13",Santa Barbara,PA,93109,34.402151,-119.722548,3.0,11,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsGo...","Restaurants, Sandwiches, Salad, Fast Food","{'Monday': '9:0-22:30', 'Tuesday': '9:0-22:30'..."
12298,8VhFYV-IZQuzWteF0qWqyg,Palermo's Pizza,509 S Broad St,Lansdale,TN,19446,40.235272,-75.288871,3.5,48,1,"{'NoiseLevel': 'u'average'', 'RestaurantsPrice...","Restaurants, Pizza",None
11604,e89rn8LPjP5BtMpCewS-TQ,Al Basha,1045 Broad Ripple Ave,Indianapolis,MO,46220,39.869705,-86.140729,4.0,8,0,"{'RestaurantsPriceRange2': '1', 'RestaurantsDe...","Restaurants, Mediterranean",None


In [18]:
# Crear una lista de IDs a partir de df1
lista_de_ids = df_business_Yelp['business_id'].tolist()

# Filtrar df2 basándose en la lista de IDs de df1
df_reviews_r = df_reviews[df_reviews['business_id'].isin(lista_de_ids)]

In [19]:
df_reviews_r.shape

(1191794, 9)

USER: Data del usuario incluyendo referencias a otros usuarios amigos y a toda la metadata asociada al usuario.

In [52]:
archivo_parquet = "../PROYECTO_FINAL_EQUIPO/Yelp/user-001.parquet"

# Abre el archivo Parquet y crea un DataFrame.

df_user = pd.read_parquet(archivo_parquet)

In [53]:
df_user.shape

(2105597, 22)

Filtrar usuarios que han reseñado restaurantes

In [70]:
#Analizar duplicados:
duplicados = df_user[df_user['user_id'].duplicated()]
print(duplicados.shape)
# Ordena los registros duplicados por 'user_id'
duplicados_ordenados = duplicados.sort_values(by='user_id')

(117700, 22)


In [72]:
#Eliminar duplicados:
df_user = df_user.drop_duplicates(subset='user_id') 

In [73]:
#Hay 1,987,897  de usuarios únicos
df_user.shape

(1987897, 22)

In [76]:
df_user

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987892,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
1987893,68czcr4BxJyMQ9cJBm6C7Q,Jane,1,2016-06-14 07:20:52,0,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
1987894,1x3KMskYxOuJCjRz70xOqQ,Shomari,4,2017-02-04 15:31:58,1,1,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
1987895,ulfGl4tdbrH05xKzh5lnog,Susanne,2,2011-01-14 00:29:08,0,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
#Ahora filtramos los usuarios únicamente que han echo alguna review
# en  los restaurantes. 
lista_de_id = df_reviews_r['user_id'].tolist()

# Filtrar basándose en la lista de IDs que aparecen en Restaurant 
df_user_review_r = df_user[df_user['user_id'].isin(lista_de_id)]

In [84]:
#Nos quedamos con 589,377 usuarios. 
df_user_review_r.shape

(589377, 22)

### Tips (consejos) escritos por el usuario. 
Los tips son más cortas que las reseñas y tienden a dar sugerencias rápidas.


In [28]:
ruta_archivo_json = "/Users/alejandrobenjaminnunezguzman/Desktop/PROYECTO_FINAL_EQUIPO/Yelp/tip.json"

lista_objetos_json = []

# Abre el archivo JSON y lee línea por línea
with open(ruta_archivo_json, "r") as archivo_json:
    for linea in archivo_json:
        try:
            # Intenta cargar la línea como un objeto JSON
            objeto_json = json.loads(linea)
            lista_objetos_json.append(objeto_json)
        except json.JSONDecodeError as e:
            print(f"Error al cargar línea: {linea}")
            continue

# Crea un DataFrame a partir de la lista de objetos JSON
df_tip = pd.DataFrame(lista_objetos_json)

In [88]:
df_tip.shape

(908915, 5)

In [90]:
df_tip.head(3)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0


In [91]:
#Ahora filtramos los usuarios que únicamente han dejado tips
#en  los restaurantes. 
df_tip_r = df_tip[df_tip['user_id'].isin(lista_de_id)]

In [92]:
#Obtenemos 640,808 tips de diferentes usuarios
df_tip_r.shape

(640808, 5)